In [1]:
import pprint, itertools
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import neighbors, metrics, model_selection
from matplotlib import pyplot as plt

In [2]:
survived_train = pd.read_csv('survived_train.csv', sep='|', index_col='PassengerId')
survived_test = pd.read_csv('survived_test.csv', sep='|', index_col='PassengerId')

train = pd.read_csv('data_train.csv', sep='|', index_col='PassengerId')
test = pd.read_csv('data_test.csv', sep='|', index_col='PassengerId')

In [3]:
survived_train.head()

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [4]:
survived_test.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [5]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,NaN
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C8,C85
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,NaN
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C1,C123
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,NaN


In [6]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,NaN,NaN
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,NaN,NaN
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,NaN,NaN
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,NaN,NaN
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,NaN,NaN


# <font color="red">Vii) Nettoyage</font>

### <font color="purple">Impact des supressions des nan</font>
#### <font color="blue">Supprimons les lignes de train qui contiennent des NaN</font>
#### <font color="blue">Cela concerne Age, Embarked, Cabin</font> 

In [7]:
clean_train_embarked = train[train['Embarked'].notnull()]

str(round(float(clean_train_embarked.shape[0])/train.shape[0]*100)) + ' %'

'100.0 %'

In [8]:
clean_test_embarked = test[test['Embarked'].notnull()]

str(round(float(clean_test_embarked.shape[0])/test.shape[0]*100)) + ' %'

'100.0 %'

#### <font color="blue">Le nettoyage avec le Embarked ne supprime pas beaucoup de données.</font>

In [9]:
clean_train_age = train[train['Age'].notnull()]

str(round(float(clean_train_age.shape[0])/train.shape[0]*100)) + ' %'

'80.0 %'

In [10]:
clean_test_age = test[test['Age'].notnull()]

str(round(float(clean_test_age.shape[0])/test.shape[0]*100)) + ' %'

'79.0 %'

#### <font color="blue">Le nettoyage avec l'âge supprime tout de même 20% des données</font> 

In [11]:
clean_train_cabin = train[train['Cabin'].notnull()]

str(round(float(clean_train_cabin.shape[0])/train.shape[0]*100)) + ' %'

'23.0 %'

In [12]:
clean_test_cabin = test[test['Cabin'].notnull()]

float(clean_test_cabin.shape[0])/test.shape[0]*100

21.770334928229666

#### <font color="blue">Le nettoyage avec Cabin ne garde que 20% des données</font>

### <font color="brown">Clean avec Age et Embarked</font>

In [13]:
clean_train_embarked_age = train[train['Age'].notnull() & train['Embarked'].notnull()]

str(round(float(clean_train_embarked_age.shape[0])/train.shape[0]*100)) + ' %'

'80.0 %'

In [14]:
clean_test_embarked_age = test[test['Age'].notnull() & test['Embarked'].notnull()]

str(round(float(clean_test_embarked_age.shape[0])/test.shape[0]*100)) + ' %'

'79.0 %'

#### <font color="blue">Le nettoyage du train avec l'age et l'embarked correspond au nettoyage du train avec l'age uniquement</font> 
#### <font color="blue">Les nan embarked sont donc quasiment tous inclus dans les nan de l'age</font> 

In [15]:
index = pd.Series(clean_train_embarked.index.isin(clean_train_age.index))

index.value_counts()/index.shape[0]*100

True     80.089989
False    19.910011
dtype: float64

In [16]:
index = pd.Series(clean_test_embarked.index.isin(clean_test_age.index))

index.value_counts()/index.shape[0]*100

True     79.425837
False    20.574163
dtype: float64

#### <font color="blue">oui les nan embarked sont à 80% inclus dans les nan Age</font> 
#### <font color="blue">on va donc conserver ce nettoyage et on verra plutard pour les numéros de cabines</font> 

In [17]:
clean_train_embarked_age.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,NaN
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C8,C85
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,NaN
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C1,C123
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,NaN


### <font color="brown">Drop avec Cabin, Bridge, Name et Ticket</font>

In [18]:
clean_train_embarked_age_drop_cabin_bridge = clean_train_embarked_age.drop(['Cabin', 'Bridge'], axis=1)

In [19]:
clean_train_embarked_age_drop_cabin_bridge.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [20]:
clean_train_embarked_age.drop(['Name', 'Ticket'], axis=1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,
1,3,male,22.0,1,0,7.2500,S,NaN,NaN
2,1,female,38.0,1,0,71.2833,C,C8,C85
3,3,female,26.0,0,0,7.9250,S,NaN,NaN
4,1,female,35.0,1,0,53.1000,S,C1,C123
5,3,male,35.0,0,0,8.0500,S,NaN,NaN
7,1,male,54.0,0,0,51.8625,S,E4,E46
8,3,male,2.0,3,1,21.0750,S,NaN,NaN
9,3,female,27.0,0,2,11.1333,S,NaN,NaN
10,2,female,14.0,1,0,30.0708,C,NaN,NaN


### <font color="brown">Age et Fare astype int</font>

In [21]:
clean_train_embarked_age_drop_cabin_bridge['Age'] = clean_train_embarked_age_drop_cabin_bridge['Age'].astype('int')
clean_train_embarked_age_drop_cabin_bridge['Fare'] = clean_train_embarked_age_drop_cabin_bridge['Fare'].astype('int')

In [22]:
clean_train_embarked_age_drop_cabin_bridge.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71,C
3,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53,S
5,3,"Allen, Mr. William Henry",male,35,0,0,373450,8,S


In [23]:
clean_train_embarked_age_drop_cabin_bridge_name_ticket = clean_train_embarked_age_drop_cabin_bridge.drop(['Name', 'Ticket'], axis=1)

In [24]:
clean_train_embarked_age_drop_cabin_bridge_name_ticket.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22,1,0,7,S
2,1,female,38,1,0,71,C
3,3,female,26,0,0,7,S
4,1,female,35,1,0,53,S
5,3,male,35,0,0,8,S


In [25]:
clean_test_embarked_age_drop_cabin_bridge_name_ticket = clean_test_embarked_age.drop(['Cabin', 'Bridge', 'Name', 'Ticket'], axis=1)
clean_test_embarked_age_drop_cabin_bridge_name_ticket['Age'] = clean_test_embarked_age_drop_cabin_bridge_name_ticket['Age'].astype('int')
# clean_test_embarked_age_drop_cabin_bridge_name_ticket['Fare'] = clean_test_embarked_age_drop_cabin_bridge_name_ticket['Fare'].astype('int')

In [26]:
type(survived_train.head())

pandas.core.frame.DataFrame

### <font color="brown">Nettoyage de Fare dans le test</font>

In [27]:
clean_test_embarked_age_drop_cabin_bridge_name_ticket['Fare'].notnull().all()

False

#### <font color="blue">Rappel Le Fare n'a pas de valeurs manquantes dans le train</font> 
#### <font color="blue">Le Fare a une valeur manquante dans le test.</font> 
#### <font color="blue">on nettoye les deux en supprimant la ligne</font> 

In [28]:
test['Fare'][test['Fare'].isnull()].index

Int64Index([1044], dtype='int64', name=u'PassengerId')

In [29]:
clean_test_embarked_age_drop_cabin_bridge_name_ticket = clean_test_embarked_age_drop_cabin_bridge_name_ticket.drop(test['Fare'][test['Fare'].isnull()].index, axis=0)

In [30]:
clean_test_embarked_age_drop_cabin_bridge_name_ticket['Fare'] = clean_test_embarked_age_drop_cabin_bridge_name_ticket['Fare'].astype('int')

In [31]:
clean_test_embarked_age_drop_cabin_bridge_name_ticket.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34,0,0,7,Q
893,3,female,47,1,0,7,S
894,2,male,62,0,0,9,Q
895,3,male,27,0,0,8,S
896,3,female,22,1,1,12,S


### <font color="brown">Définition des sets de base Drop cabin name ticket bridge</font>

In [32]:
train_final = clean_train_embarked_age_drop_cabin_bridge_name_ticket
test_final = clean_test_embarked_age_drop_cabin_bridge_name_ticket

In [33]:
train_final.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22,1,0,7,S
2,1,female,38,1,0,71,C
3,3,female,26,0,0,7,S
4,1,female,35,1,0,53,S
5,3,male,35,0,0,8,S


In [34]:
test_final.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34,0,0,7,Q
893,3,female,47,1,0,7,S
894,2,male,62,0,0,9,Q
895,3,male,27,0,0,8,S
896,3,female,22,1,1,12,S


In [35]:
survived_train_final = survived_train.loc[train_final.index]
survived_test_final = survived_test.loc[test_final.index]

In [36]:
survived_train_final.head()

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [37]:
survived_test_final.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [38]:
print 'Train : ' + str(round(float(train_final.shape[0])/train.shape[0]*100)) + ' %'
print 'Test : ' + str(round(float(test_final.shape[0])/test.shape[0]*100)) + ' %'

Train : 80.0 %
Test : 79.0 %


## <font color="purple">Sauvegarde</font>

In [39]:
train_final.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/clean_train_embarked_age_drop_cabin_bridge_name_ticket.csv', sep='|')
test_final.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/clean_test_embarked_age_drop_cabin_bridge_name_ticket.csv', sep='|')
survived_train_final.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/survived_train_final.csv', sep='|')
survived_test_final.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/survived_test_final.csv', sep='|')